In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Rescaling, Conv2D, MaxPooling2D, Flatten, Dense, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy
import os
import cv2
import matplotlib.pyplot as plt

dataset: https://www.kaggle.com/ivanver/notebookafce69a0eb/data

In [ ]:
MAIN_PATH = "../input/brain-mri-images-for-brain-tumor-detection/brain_tumor_dataset/"
IMG_WIDTH = 256
IMG_HEIGHT = 256
os.listdir(MAIN_PATH)

In [ ]:
images_list = os.listdir(MAIN_PATH + "/no/")
for i in images_list[:2]:
    img = cv2.imread(MAIN_PATH + "/no/" + i)
    fig, ax = plt.subplots()
    ax.imshow(img)
   

In [ ]:
images_list = os.listdir(MAIN_PATH + "/yes/")
for i in images_list[:2]:
    img = cv2.imread(MAIN_PATH + "/yes/" + i)
    fig, ax = plt.subplots()
    ax.imshow(img)
   

In [ ]:
train_img_ds = tf.data.Dataset.list_files(MAIN_PATH + '*/' + '*.jpg', shuffle=True)

In [ ]:
def preprocess_img(file_path):
    img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    img = tf.cast(img, tf.float32) / 255.0
    parts = tf.strings.split(file_path, os.path.sep)[-2]
    label = [int(tf.equal(i, parts)) for i in ['no', 'yes']]
    return img, label

In [ ]:
train_img_ds = train_img_ds.map(preprocess_img)
train_img_ds = train_img_ds.batch(40).prefetch(1)

In [ ]:
for i in train_img_ds:
    print(i[1].shape)
    break

In [ ]:
model = Sequential([
    Input(shape=(256, 256, 3)),
    Conv2D(16, 3, padding='same', kernel_initializer="he_normal", activation='relu'),
    Conv2D(16, 3, padding='same', kernel_initializer="he_normal", activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', kernel_initializer="he_normal", activation='relu'),
    Conv2D(32, 3, padding='same', kernel_initializer="he_normal", activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', kernel_initializer="he_normal", activation='relu'),
    Conv2D(64, 3, padding='same', kernel_initializer="he_normal", activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 3, padding='same', kernel_initializer="he_normal", activation='relu'),
    Conv2D(128, 3, padding='same', kernel_initializer="he_normal", activation='relu'),
    MaxPooling2D(),
    Conv2D(256, 3, padding='same', kernel_initializer="he_normal", activation='relu'),
    Conv2D(256, 3, padding='same', kernel_initializer="he_normal", activation='relu'),
    MaxPooling2D(),
    Conv2D(512, 3, padding='same', kernel_initializer="he_normal", activation='relu'),
    Conv2D(512, 3, padding='same', kernel_initializer="he_normal", activation='relu'),
    MaxPooling2D(),
    Conv2D(1024, 3, padding='same', kernel_initializer="he_normal", activation='relu'),
    Conv2D(1024, 3, padding='same', kernel_initializer="he_normal", activation='relu'),
    MaxPooling2D(),
    Conv2D(2048, 3, padding='same', kernel_initializer="he_normal", activation='relu'),
    Conv2D(2048, 3, padding='same', kernel_initializer="he_normal", activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(1024, activation='relu', kernel_initializer="he_normal"),
    Dense(1024, activation='relu', kernel_initializer="he_normal"),
    Dense(1024, activation='relu', kernel_initializer="he_normal"),
    Dense(1024, activation='relu', kernel_initializer="he_normal"),
    Dense(512, activation='relu', kernel_initializer="he_normal"),
    Dense(512, activation='relu', kernel_initializer="he_normal"),
    Dense(512, activation='relu', kernel_initializer="he_normal"),
    Dense(512, activation='relu', kernel_initializer="he_normal"),
    Dense(512, activation='relu', kernel_initializer="he_normal"),
    Dense(256, activation='relu', kernel_initializer="he_normal"),
    Dense(256, activation='relu', kernel_initializer="he_normal"),
    Dense(256, activation='relu', kernel_initializer="he_normal"),
    Dense(128, activation='relu', kernel_initializer="he_normal"),
    Dense(64, activation='relu', kernel_initializer="he_normal"),
    Dense(32, activation='relu', kernel_initializer="he_normal"),
    Dense(2, activation='softmax')
])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss=BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_img_ds, 
    epochs=60
)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
no_img_list = "../input/brain-mri-images-for-brain-tumor-detection/no/"
yes_img_list = "../input/brain-mri-images-for-brain-tumor-detection/yes/"

In [ ]:
no_img_list_dir = os.listdir(MAIN_PATH + 'no/')
yes_img_list_dir = os.listdir(MAIN_PATH + 'yes/')

In [ ]:
def proccess(dir):
    img = cv2.imread(dir)
    img = cv2.resize(img, (256, 256), interpolation = cv2.INTER_AREA)
    return img / 255.0 

In [ ]:
no_img_list_array = np.array([proccess(no_img_list + i) for i in no_img_list_dir])
yes_img_list_array = np.array([proccess(yes_img_list + i) for i in yes_img_list_dir])

In [ ]:
result_no = model.predict(no_img_list_array[:20])
result_yes = model.predict(yes_img_list_array[:20])

In [ ]:
np.round(result_no)

In [ ]:
np.round(result_yes)